In [ ]:
library(arrow)
library(dplyr)
library(mgcv)
library(tidyverse)
library(data.table)
library(metagam)
library(latex2exp)

In [ ]:
path <- "metadata_mean.parquet"
df <- arrow::read_parquet(path)
path <- "metadata_raw.parquet"
dfraw <- arrow::read_parquet(path)

In [ ]:
df = df %>% dplyr::select(-'__index_level_0__') %>% mutate(siteyr=paste0(site,year)) %>% filter(Importances!=0)
siteyrlist = df %>% dplyr::select(siteyr) %>% unique
featuredecodelist = df %>% dplyr::select(featuredecode) %>% unique

In [ ]:
# library('fda.usc')
# # fregre.basis
# data(tecator)
# names(tecator)
# absorp=tecator$absorp.fdata
# ind=1:129
# x=absorp[ind,]
# y=tecator$y$Fat[ind]
# tt=absorp[["argvals"]]
# res1=fregre.basis(x,y)
# summary(res1)
# basis1=create.bspline.basis(rangeval=range(tt),nbasis=19)
# basis2=create.bspline.basis(rangeval=range(tt),nbasis=9)
# res5=fregre.basis(x,y,basis1,basis2)
# summary(res5)
# x.d2=fdata.deriv(x,nbasis=19,nderiv=1,method="bspline",class.out="fdata")
# res7=fregre.basis(x.d2,y,basis1,basis2)
# summary(res7)

In [ ]:
plot_fit<-function(df, fit1){
    df2<-df %>%
        mutate(lwr = mean_val - 2 * se_val,
         upr = mean_val + 2 * se_val)
    
#    fit1_FIT<-predict(fit1$gam, se.fit=T) %>%
#    newd <- data.frame(fval=0:600)
    fit1_FIT<-predict(fit1, se.fit=T) %>%    
#    fit1_FIT<-predict(fit1, newd, se.fit=T) %>%        
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)
    
    ggplot()+
    geom_point(aes(df2$fval, df2$mean_val))+
    geom_line(aes(df2$fval, fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2$fval, ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")+
#    geom_line(aes(newd$fval, fit1_FIT$fit_gam, color='red'))+
#    geom_ribbon(aes(x=newd$fval, ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")+    
    geom_errorbar(aes(x=df2$fval, ymin=df2$lwr, ymax=df2$upr), alpha = 0.2)
}

In [ ]:
plot_fit2<-function(df2, gamobj, coeff){
    
#    df2<-df2 %>%
#        mutate(lwr = df2[[coeff]] - 2 * df2[[paste0(coeff,'_var')]],
#         upr = df2[[coeff]] + 2 * df2[[paste0(coeff,'_var')]])    
   
    fit1_FIT<-predict(gamobj, se.fit=T) %>%
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)

    ggplot()+
    geom_point(aes(df2[[coeff]], df2$auc))+
    geom_line(aes(df2[[coeff]], fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2[[coeff]], ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")
#    geom_errorbarh(aes(y=df2$auc, xmin=df2$lwr, xmax=df2$upr), alpha = 0.2)    
}

In [ ]:
# drop term (17<AGE<90, other 3sigma)
# filterdf <- df %>% 
#     select('featuredecode', 'fval') %>% 
#     group_by(featuredecode) %>% 
#     summarise(mean=mean(fval), sd=sd(fval)) %>% 
#     mutate(upr=mean+3*sd, lwr=mean-3*sd) %>% 
#     mutate(upr = replace(upr, featuredecode == "AGE", 89)) %>% 
#     mutate(lwr = replace(lwr, featuredecode == "AGE", 18)) %>%
#     select('featuredecode', 'upr', 'lwr')
filterdf <- df %>% 
    select(site,year,featuredecode,fval) %>% 
    group_by(site,year,featuredecode) %>% 
    summarise(min=min(fval), max=max(fval)) %>%
    ungroup %>%
    group_by(featuredecode) %>%
    summarise(min_min=min(min), max_min=max(min), min_max=min(max), max_max=max(max), mean_min=mean(min), sd_min=sd(min), mean_max=mean(max), sd_max=sd(max)) %>%
    mutate(upr=mean_max+3*sd_max, lwr=mean_min-3*sd_min) %>% 
    mutate(upr = replace(upr, featuredecode == "AGE", 89)) %>% 
    mutate(lwr = replace(lwr, featuredecode == "AGE", 18))

df <- df %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)
filterdf

In [ ]:
#Check the min max outliner drop

# dfplot<- df %>% 
#     select(site,year,featuredecode,fval,siteyr) %>% 
#     group_by(siteyr,featuredecode) %>% 
#     summarise(min=min(fval), max=max(fval)) %>%
#     ungroup %>% filter(featuredecode=='2075-0')
# barplot(dfplot$min, names.arg=dfplot$siteyr)

In [ ]:
# imputation using metagam

fits <- list()
for (fc in featuredecodelist$featuredecode){
#    fc <- 'SYSTOLIC'
    dft = df %>% filter(featuredecode==fc)
    maxx <- dft %>% select('fval') %>% max()
    minx <- dft %>% select('fval') %>% min()    
    for (sy in siteyrlist$siteyr){
        dft <- df %>% filter(siteyr==sy) 
        dft_fit <- list(gam(mean_val ~ s(fval, k=60, bs='cr'), data=dft%>%select(fval,mean_val), method='REML'))
        names(dft_fit) <- paste0(sy,'_',fc)
        fits <- append(fits, dft_fit)
    }
}


# cohort_fits <- map(fits, strip_rawdata, save_tanges=TRUE)
# grid <- tibble(fval = seq(from=minx, to=maxx,by=((maxx-minx)/1000)))

# pred_data <- imap_dfr(cohort_fits, function(fit, cohort){

#     lower <- min(fit$var.summary$fval)
#     upper <- max(fit$var.summary$fval)

#     grid2 <- grid %>%
#     filter(fval > !!lower, fval < !! upper)

#     pred <- predict(fit, newdata = grid2, se.fit = TRUE,
#     type = "iterms", terms = c("s(fval)"))

#     grid2 %>%
#     mutate(
#     fit = rowSums(pred$fit) + attr(pred, "constant"),
#     se = apply(pred$se.fit, 1, function(x) sqrt(sum(x^2))),
#     cohort = cohort
#     )
# })
# p <- ggplot(pred_data, aes(x = fval, y = fit, ymin = fit + qnorm(.025) * se,
#             ymax = fit + qnorm(.975) * se)) +
#             geom_line() +
#             geom_ribbon(alpha = .3, color = NA) +
#             facet_wrap(vars(cohort)) +
#             theme_classic() +
#             theme(strip.background = element_blank(),
#             panel.border = element_rect(colour = "black", fill = NA)) +
#             ylab(TeX("s(fval) (mm$^{3}$)"))
# #p
# metafit <- metagam(cohort_fits, grid, type = "iterms", terms = "s(fval)", method = "DL", intercept = TRUE)

In [ ]:
dftt<-df%>%filter(siteyr=='MCRI2014')%>%filter(featuredecode=='SYSTOLIC')
vi <- 1/dftt$se_val
dftt_fit <- gam(mean_val ~ s(fval, k=60, bs='cr'), data=dftt, method='REML')         
#plot_fit(dftt,dftt_fit)
dftt$fval

In [ ]:
dftt$se_val

In [ ]:
fitlist<-list()
df2<-list()
for (x in siteyrlist$siteyr){
#    print(x)
    df2[[x]]<-df %>% 
    filter(siteyr==x) %>% 
    dplyr::select('mean_val', 'fval', 'se_val') 
    fitlist[[x]]<-gam(mean_val ~ s(fval, k=10, bs='ps'), data=df2[[x]], method='REML')
#    print(paste0('finished ',x))
}

In [ ]:
plots<-list()
for (x in siteyrlist$siteyr){
#    print(k.check(x$gam))}
    plots[[x]]<-plot_fit(df2[[x]], fitlist[[x]])
#    ggsave(paste0('plots/AGE_',x,'.svg'),plot=plots[[x]],device='svg')
    print(plots[[x]])
}

In [ ]:
coeffdata <- t(data.frame(fitlist[['KUMC2010']]$coefficients))

for (x in siteyrlist$siteyr){
    row1 <- t(data.frame(fitlist[[x]]$coefficients))
    rownames(row1) <- c(x)
    coeffdata = rbind(coeffdata,row1)
}
coeffdata=coeffdata[-1,]
colnames(coeffdata) = str_remove_all(colnames(coeffdata),'[.()]')
metaformula<-as.formula(paste0("auc~", paste(colnames(coeffdata)[c(-1)], collapse = '+')))
auc <- df %>% dplyr::select('siteyr', 'auc') %>% unique
coeffdata <- tibble::rownames_to_column(data.frame(coeffdata), "siteyr")
coeffdata <- coeffdata %>% left_join(auc, by='siteyr')

In [ ]:
coeffvardata <- t(data.frame(diag(fitlist[['KUMC2010']]$Vc)))

for (x in siteyrlist$siteyr){
    row1 <- t(data.frame(diag(fitlist[[x]]$Vc)))
    rownames(row1) <- c(x)
    coeffvardata = rbind(coeffvardata,row1)
}
coeffvardata=coeffvardata[-1,]
colnames(coeffvardata) = lapply(colnames(coeffdata)[-length(colnames(coeffdata))][-1], function(x) paste0(x,'_var'))
coeffvardata <- tibble::rownames_to_column(data.frame(coeffvardata), "siteyr")

In [ ]:
coeffdata_wvar <- coeffdata %>% left_join(coeffvardata, by='siteyr')

In [ ]:
metagame<-gam(metaformula, data=coeffdata, method='REML')

In [ ]:
summary(metagame)

In [ ]:
f2<- auc ~ s(Intercept, k=60, bs='cr') + s(sfval1, k=60, bs='cr')

In [ ]:
metagame3<-gam(auc ~ sfval6, data=coeffdata, method='REML')

In [ ]:
summary(metagame3)

In [ ]:
plot_fit2(coeffdata_wvar, metagame3, 'sfval6')

In [ ]:
summary(fitlist[['KUMC2010']])

In [ ]:
??eivtools

In [ ]:
library(arrow)
library(dplyr)
library(mgcv)
library(tidyverse)
library(data.table)
library(metagam)
library(latex2exp)
library(eivtools)
library(plotly)
library(stats)
library(plotrix)
library(fishmethods)
library(metafor)

In [ ]:
plot_fit2<-function(df2, gamobj, coeff){
    
    df2<- df2 %>% mutate(site = substr(siteyr,1,nchar(siteyr)-4))
    #    df2<-df2 %>%
    #        mutate(lwr = df2[[coeff]] - 2 * df2[[paste0(coeff,'_var')]],
    #         upr = df2[[coeff]] + 2 * df2[[paste0(coeff,'_var')]])    

    fit1_FIT<-predict(gamobj, se.fit=T) %>%
      as_tibble() %>% 
      rename(fit_gam = fit) %>% 
      mutate(lwr_gam = fit_gam - 2 * se.fit,
             upr_gam = fit_gam + 2 * se.fit)

    ggplot()+
    geom_point(aes(df2[[coeff]], df2$auc, color=df2$site))+
    geom_line(aes(df2[[coeff]], fit1_FIT$fit_gam, color='red'))+
    geom_ribbon(aes(x=df2[[coeff]], ymin=fit1_FIT$lwr_gam, ymax=fit1_FIT$upr_gam), alpha = 0.2, fill = "red")
#    geom_errorbarh(aes(y=df2$auc, xmin=df2$lwr, xmax=df2$upr), alpha = 0.2)    
}

In [ ]:
path <- "metadata_mean.parquet"
df <- arrow::read_parquet(path)
path <- "metadata_raw.parquet"
dfraw <- arrow::read_parquet(path)

In [ ]:
df = df %>% dplyr::select(-'__index_level_0__') %>% mutate(siteyr=paste0(site,year)) %>% filter(Importances!=0)

#df = df %>% filter(!(site %in% list("MCRI"))) %>% filter(!(sy %in% list("UMHC2013", "UMHC2020")))
#df = df %>% filter(!(site %in% list("MCRI", 'UPITT'))) %>% filter(!(siteyr %in% list("UMHC2020")))
df = df %>% filter(!(siteyr %in% list("UMHC2020")))

siteyrlist = df %>% dplyr::select(siteyr) %>% unique
featuredecodelist = df %>% dplyr::select(featuredecode) %>% unique
dfraw = dfraw %>% rename(fval=Name) %>% rename(siteyr=siteyear) %>% dplyr::select(-'__index_level_0__')

In [ ]:
filterdf <- df %>% 
    select(site,year,featuredecode,fval) %>% 
    group_by(site,year,featuredecode) %>% 
    summarise(min=min(fval), max=max(fval)) %>%
    ungroup %>%
    group_by(featuredecode) %>%
    summarise(min_min=min(min), max_min=max(min), min_max=min(max), max_max=max(max), mean_min=mean(min), sd_min=sd(min), mean_max=mean(max), sd_max=sd(max)) %>%
    mutate(upr=mean_max+3*sd_max, lwr=mean_min-3*sd_min) %>% 
    mutate(upr = replace(upr, featuredecode == "AGE", 89)) %>% 
    mutate(lwr = replace(lwr, featuredecode == "AGE", 18))

df <- df %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)
dfraw <- dfraw %>% left_join(filterdf%>%select(featuredecode,upr,lwr), by='featuredecode') %>% filter(fval>=lwr) %>% filter(fval<=upr) %>% select(-upr, -lwr)

In [ ]:
frange <- dfraw %>% select(fval, featuredecode) %>% group_by(featuredecode) %>% 
                        summarize(min=min(fval),q10 =quantile(fval,probs=0.001),
                                                q25 =quantile(fval,probs=0.25),
                                                q375=quantile(fval,probs=0.375),
                                                q50 =quantile(fval,probs=0.50),
                                                q625=quantile(fval,probs=0.625),
                                                q75 =quantile(fval,probs=0.75),
                                                q90 =quantile(fval,probs=0.999),
                                                max=max(fval))
frange

In [ ]:
library(foreach)
library(doParallel)
ptm <- proc.time()
gamdict <- list()
fitlist <- list()
#sy <- 'KUMC2013'
#syf <- 'SYSTOLIC'
cl <- parallel::makeCluster(16)
doParallel::registerDoParallel(cl)

In [ ]:
syflistdict <- dfraw %>% select(siteyr, featuredecode) %>% unique

In [ ]:
#Alternatively fit and get coefficient

#Make Average to be zero?
#dfraw <- dfraw %>% group_by(siteyr, Feature) %>% mutate(val=val-mean(val)) %>% ungroup

ptm <- proc.time()
gamdict <- list()
fitlist <- list()
#sy <- 'KUMC2013'
#syf <- 'SYSTOLIC'
test<-foreach (srrow=iter(syflistdict, by='row'),
              .combine=rbind) %dopar% {
        flush.console()
        return(srrow)
}
test

In [ ]:
ptm <- proc.time()
fitlist<-foreach (srrow=iter(syflistdict, by='row'), .combine=rbind) %do% {
        library(arrow)
        library(dplyr)
        library(mgcv)
        library(tidyverse)
        library(data.table)
        library(metagam)
        library(latex2exp)
        library(eivtools)
        library(plotly)
        library(stats)
        library(plotrix)
        library(fishmethods)
        library(metafor)    
        sy <- srrow$siteyr
        syf <- srrow$featuredecode
        print(paste0(sy,'-',syf))
        flush.console()
        dfrawsy <- dfraw %>% filter(siteyr==sy)
        syflist <- dfrawsy %>% select(featuredecode) %>% unique    
        flush.console()
        dftt <- dfrawsy %>% filter(featuredecode==syf)
        fr<-frange %>% filter(featuredecode==syf)
        if (!all(dftt['val']==0)){
            x<-list(gam(val ~ s(fval, k=6, bs='cr'), data=dftt, method='REML', knots = list(fval = seq(from=fr$min, to=fr$max, length = 6))))
            names(x) <- syf
#            gamsydict <- append(gamsydict,x)
            
            rangelist <- list()
            for (i in 1:length(x[[syf]]$smooth[[1]]$xp)){
                rangelist <- c(rangelist, paste0('r', x[[syf]]$smooth[[1]]$xp[i], 'r', x[[syf]]$smooth[[1]]$xp[i+1], 'r'))
            }
            rangelist <- rangelist[-length(rangelist)]
            rangelist <- c('intercept', rangelist)
            coeffdata <- cbind(data.frame(x[[syf]]$coefficient), data.frame(diag(x[[syf]]$Vc)), data.frame(transpose(rangelist)))
            colnames(coeffdata) <- c('fit', 'se.fit', 'fval')
            coeffdata <- coeffdata %>% mutate(siteyr=sy,featuredecode=syf)
            coeffdata <- coeffdata[-1,]
            coeffdata <- coeffdata %>% mutate(fval=paste0(featuredecode,'.',fval))
            row.names(coeffdata) <- NULL
#            fitlist <- append(fitlist,list(coeffdata))            
            return(coeffdata)
        }
    }
#    gamsydict2<-list(gamsydict)
#    names(gamsydict2) <- sy
#    gamdict <- append(gamdict,gamsydict2)

proc.time() - ptm
#gamdict<-rbindlist(gamdict)
#fitlist<-rbindlist(fitlist) %>% mutate(featuredecode=str_replace(featuredecode,'-','d')) %>% mutate(featuredecode=paste0('f',featuredecode)) %>% mutate(fval=str_replace_all(fval,'-','d')) %>% mutate(fval=paste0('f',fval))

In [ ]:
fitlist<- fitlist %>% mutate(featuredecode=str_replace(featuredecode,'-','d')) %>% mutate(featuredecode=paste0('f',featuredecode)) %>% mutate(fval=str_replace_all(fval,'-','d')) %>% mutate(fval=paste0('f',fval))
fitlist

In [ ]:
stopCluster(myCluster)